# Project
## 获取数据


In [3]:
import tensorflow as tf
import matplotlib.pylab as plt
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import itertools



import paramiko
from stat import S_ISDIR
import os



# 定义一个类，表示一台远端linux主机
class Linux(object):
    # 通过IP, 用户名，密码，超时时间初始化一个远程Linux主机
    def __init__(self, ip, username, password, timeout=30):
        self.ip = ip
        self.username = username
        self.password = password
        self.timeout = timeout
        # transport和chanel
        self.t = ''
        self.chan = ''
        # 链接失败的重试次数
        self.try_times = 3

    # 调用该方法连接远程主机
    def connect(self):
         pass

    # 断开连接
    def close(self):
        pass

    # 发送要执行的命令
    def send(self, cmd):
        pass

    # get单个文件
    def sftp_get(self, remotefile, localfile):
        t = paramiko.Transport(sock=(self.ip, 22))
        t.connect(username=self.username, password=self.password)
        sftp = paramiko.SFTPClient.from_transport(t)
        sftp.get(remotefile, localfile)
        t.close()

    # put单个文件
    def sftp_put(self, localfile, remotefile):
        t = paramiko.Transport(sock=(self.ip, 22))
        t.connect(username=self.username, password=self.password)
        sftp = paramiko.SFTPClient.from_transport(t)
        sftp.put(localfile, remotefile)
        t.close()
        
    def __get_all_files_in_remote_dir(self, sftp, remote_dir):
        #保存文件夹文件的列表
        all_files = list()

        #去掉最后的/如果有
        if remote_dir[-1] == "/":
            remote_dir = remote_dir[0:-1]

        #获取目录下的所有文件
        files = sftp.listdir_attr(remote_dir)
        for x in files:
            # remote_dir目录中每一个文件或目录的完整路径
            filename = remote_dir + '/' + x.filename
            # 如果是目录，则递归处理该目录，
            # 这里用到了stat库中的S_ISDIR方法，与linux中的宏的名字完全一致
            if S_ISDIR(x.st_mode):
                all_files.extend(self.__get_all_files_in_remote_dir(sftp, filename))
            else:
                all_files.append(filename)
        return all_files 
        
    def sftp_get_dir(self, remote_dir, local_dir):
        t = paramiko.Transport(sock=(self.ip, 22))
        t.connect(username=self.username, password=self.password)
        sftp = paramiko.SFTPClient.from_transport(t)

         # 获取远端linux主机上指定目录及其子目录下的所有文件
        all_files = self.__get_all_files_in_remote_dir(sftp, remote_dir)
        # 依次get每一个文件
        for x in all_files:
            filename = x.split('/')[-1]
            local_filename = os.path.join(local_dir, filename)
            print('loading file {}, please wait'.format(filename) )
            sftp.get(x, local_filename)
        
            
    def sftp_remove_dir(self, remote_dir):
        t = paramiko.Transport(sock=(self.ip, 22))
        t.connect(username=self.username, password=self.password)
        sftp = paramiko.SFTPClient.from_transport(t)

        
         # 获取远端linux主机上指定目录及其子目录下的所有文件
        all_files = self.__get_all_files_in_remote_dir(sftp, remote_dir)
        # 依次remove每一个文件
        
        for x in all_files:
            filename = x.split('/')[-1]
            
            print('removing file {}, please wait'.format(filename) )
            sftp.remove(x)


### 设置IP和路径

In [103]:
remote_path = '/home/pi/camera capture'
local_path = "/Users/allen/Desktop/tensorflow/database/pictures"
host = Linux('10.0.0.164', 'pi', '123456')

### 下载树莓派中所有路径下的文件到本地文件夹

In [105]:
host.sftp_get_dir(remote_path, local_path)

loading file 2020_01_16_18_51_23%%182g.jpg, please wait
loading file 2020_01_16_18_51_36%%158g.jpg, please wait


### 清空远程文件夹

In [ ]:
host.sftp_remove_dir(remote_path)

## 神经网络数据的处理

### 加载模型

In [135]:
path_5h = "/Users/allen/Desktop/Project/trained_model/MobileNet Model_my dataset_epochs = 3.h5"

model = tf.keras.models.load_model(path_5h,custom_objects={'KerasLayer':hub.KerasLayer})

### 加载数据 & 建立名字和index的关系

In [136]:
im_gen = ImageDataGenerator(rescale=1/255)# 这个generate每次调用只遍历一次，一个epoch
predict_data = im_gen.flow_from_directory("/Users/allen/Desktop/Project/Pictures/",
                                                 target_size=(224, 224),
                                                 batch_size=1,shuffle = "false",
                                                )
class_names = np.array(['Apple','Eggplant','Lemon','Mango','Orange','Peach','Pear','Pepper Green','Strawberry','Tomato','Apple_Pie','Chicken_Wings','Chocolate_Cake','Dumplings','Fish_And_Chips','French_Fries','Hamburger','Hot_Dog','Ice_Cream','Pancakes','Pizza','Steak','Sushi','Waffles'])
name_decode = {i:class_names[i] for i in range(len(class_names))}

Found 20 images belonging to 2 classes.


### 批量预测

In [143]:
import numpy as np
predicted_result = model.predict(predict_data) # 预测，
predicted_index_labels = np.argmax(predicted_result, axis=-1)  # index_label
# print(class_names[np.argmax(predicted_result, axis=-1)])# one-hot中最大的index拿出来，对应上class，name里按顺序放好的name
cnt_dic = {}
for index_label in predicted_index_labels:
    cnt_dic[name_decode[index_label]] = cnt_dic.get(name_decode[index_label],0)+1 
print(cnt_dic)

{'Mango': 1, 'Pizza': 1, 'Hamburger': 3, 'Orange': 2, 'Chicken_Wings': 2, 'Pepper Green': 1, 'Ice_Cream': 5, 'Fish_And_Chips': 1, 'Tomato': 2, 'Apple': 2}


In [142]:
import numpy as np
predicted_result = model.predict(predict_data) # 预测，
predicted_index_labels = np.argmax(predicted_result, axis=-1)  # index_label
print(predicted_index_labels)

[ 3 20 16  4 11  7 18 16 18 18  4 14  9 18 16 18  0  9  0 11]
